In [3]:
using NBInclude;
using Plots; default(fmt=:png,ls=:auto);
@nbinclude("deal_features.ipynb");

human_readable (generic function with 1 method)

In [2]:
function bid!(Q,deal,database_state,database_target,epsilon_greedy;turn_depth=6)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    current_bid=1;
    bidding_hist[1]=1;
    loss=[];
    for t in 1:turn_depth
        if t==1
            state=atype(instance[(t-1)%2+1]);
        else
            state=atype(vcat(instance[(t-1)%2+1],bidding_hist));  
        end
        target=determine_target(deal[3][:,(t-1)%2+1],current_bid); #Algorithm P
        
        #target[1]=0.1; to avoid double pass
        #exploration
        action,q_values=explore(Q[t],state,current_bid,epsilon_greedy);
        
        
        #Data for experience replay
        
        # current bid is pass----- 
        push!(loss,sum((q_values-reshape(target,(36,1))).^2))
        if action== current_bid || t==turn_depth
            target=deepcopy(deal[3][:,(t-1)%2+1]);
            push!(database_state[t],state);
            push!(database_target[t],target);  
            return target[action],loss
            break;
        end
        push!(database_state[t],state);
        push!(database_target[t],target);  
        #Update bidding history
        bidding_hist[action]=1;
        current_bid=action;
    end   
end
function bid_to_the_end(Q,deal_set,number_of_deals)
    database_state=[[],[],[],[],[],[]];
    database_target=[[],[],[],[],[],[]];
    for i in 1:number_of_deals
        bid!(Q,deal_set[rand(1:end)],database_state,database_target,0);
    end
    return database_state, database_target;
end

bid_to_the_end (generic function with 1 method)

In [3]:
function bid(Q,deal;debug=false)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    q_values_data=[];
    target_data=[];
    current_bid=1;
    bidding_hist[1]=1;
    for t in 1:turn_depth
        if debug
            println(string(t," bid"));
        end
        if t==1
            state=atype(instance[(t-1)%2+1]);
        else
            state=atype(vcat(instance[(t-1)%2+1],bidding_hist));  
        end
        target=reshape(determine_target(deal[3][:,(t-1)%2+1],current_bid),(36,1)); #Algorithm P
        #target[1]=0.1;
        
        #Data for experience replay
        #exploration
        action,q_values=explore(Q[t],state,current_bid,0);
        # current bid is pass----- 
        if action== current_bid || t==6
            target=deepcopy(deal[3][:,(t-1)%2+1]);
            if debug
                print("Bid"); print("\t"); print("Q value"); print("\t \t"); print("Cost");  print("\t"); println("Loss")
                for i in 1:36
                    print(bids[i]); print("\t"); print(q_values[i]);  print("\t"); print(target[i]); print("\t"); println(q_values[i]-target[i])
                end
            end
            push!(q_values_data,q_values)
            push!(target_data,target)
            break;
        end
        if debug
            print("Bid"); print("\t"); print("Q value"); print("\t \t"); print("Cost");  print("\t"); println("Loss")
            for i in 1:36
                print(bids[i]); print("\t"); print(q_values[i]);  print("\t"); print(target[i]); print("\t"); println(q_values[i]-target[i])
            end
            
        end    
        push!(q_values_data,q_values)
        push!(target_data,target)
        #Update bidding history
        bidding_hist[action]=1;
        current_bid=action;
    end   


    return deal[1],deal[2],bidding_hist,q_values_data,target_data
end

bid (generic function with 1 method)

In [1]:
function sample_bid(Q,deal;debug=false)
    n,s,bidding,q_values,target=bid(Q,deal,debug=debug);
    
    for i in 1:length(target)
        q_values[i]=reshape(q_values[i],(36,1));
        target[i]=reshape(target[i],(36,1));
        println("",mean((q_values[i]-atype(target[i])).^2));
    end
    println(human_readable(n));
    println(human_readable(s));
    for i in 1:36
        if bidding[i]==1
            println(bids[i]);
        end
    end 
end

sample_bid (generic function with 1 method)